In [1]:
import os
os.chdir('/content/drive/MyDrive/DataML/HyperspectralData')

In [2]:
!ls

'Create Final Dataset.ipynb'   Zero_class_classifier.ipynb
 test_set.pt		       zero_label_class.pt
 train_set.pt		       zero_label.pt


In [25]:
import torch
from torch.utils.data import Dataset
from collections import namedtuple

In [26]:
train_test_tuple = namedtuple('train_test_tuple', ['label','train_set','test_set'])

class HyperDataset(Dataset):

  def __init__(self, train_test_tuples, train = True):
    self.y = []
    self.X = []
    self.train = train
    for train_test_tuple in train_test_tuples:
      X_temp = train_test_tuple.train_set if train else train_test_tuple.test_set
      self.X.append(X_temp)
      self.y.extend([train_test_tuple.label for i in range(X_temp.shape[0])])
    self.y = torch.Tensor(self.y)
    self.X = torch.cat(self.X)
  
  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, ndx):
    return tuple([self.X[ndx,:], self.y[ndx]])

In [27]:
zero_label_matrix = torch.load('zero_label.pt')
zero_label_classes = torch.load('zero_label_class.pt')

In [28]:
zero_label_matrix.shape, zero_label_classes.shape

(torch.Size([10776, 25, 25, 30]), torch.Size([10776]))

In [29]:
# 1 -- test, 0 -- train

In [30]:
train_set = zero_label_matrix[zero_label_classes == 0]
train_set.shape

torch.Size([2276, 25, 25, 30])

In [31]:
test_set = zero_label_matrix[zero_label_classes == 1]
test_set.shape

torch.Size([8500, 25, 25, 30])

In [32]:
test_set_full = torch.load('test_set.pt')
train_set_full = torch.load('train_set.pt')

In [33]:
test_set_full.X.shape, train_set_full.X.shape

(torch.Size([7354, 25, 25, 30]), torch.Size([2895, 25, 25, 30]))

In [18]:
## adding zero_label

In [34]:
test_set_full.X = torch.cat([test_set_full.X, test_set])


In [35]:
train_set_full.X = torch.cat([train_set_full.X, train_set])

In [36]:
test_set_full.X.shape, train_set_full.X.shape

(torch.Size([15854, 25, 25, 30]), torch.Size([5171, 25, 25, 30]))

In [37]:
assert 5171 == 2895 + 2276
assert 15854 == 7354 + 8500

In [41]:
test_set_full.y.shape

torch.Size([7354])

In [45]:
test_set_full.y.reshape(-1,1)

tensor([[ 1.],
        [ 1.],
        [ 1.],
        ...,
        [16.],
        [16.],
        [16.]])

In [51]:
label_test = [1 for i in range(test_set.shape[0])]
label_test = torch.Tensor(label_test).reshape(-1,1)

In [61]:
label_train = [0 for i in range(train_set.shape[0])]
label_train = torch.Tensor(label_train).reshape(-1,1)

In [62]:
test_set_full.y = test_set_full.y.reshape(-1,1)
train_set_full.y = train_set_full.y.reshape(-1,1)

In [63]:
torch.cat([label_test, test_set_full.y]).shape, torch.cat([label_train, train_set_full.y]).shape

(torch.Size([15854, 1]), torch.Size([5171, 1]))

In [64]:
train_set_full.y = torch.cat([label_train, train_set_full.y])
test_set_full.y = torch.cat([label_test, test_set_full.y])

In [65]:
(train_set_full.X.shape, train_set_full.y.shape),(test_set_full.X.shape, test_set_full.y.shape)

((torch.Size([5171, 25, 25, 30]), torch.Size([5171, 1])),
 (torch.Size([15854, 25, 25, 30]), torch.Size([15854, 1])))

In [66]:
torch.save(train_set_full, 'final_train_set.pt')
torch.save(test_set_full, 'final_test_set.pt')